In [58]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import copy
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [59]:
tickers = ["AMZN","MSFT","INTC","GOOG", "ES"]
start = dt.datetime.today() - dt.timedelta(700)
end = dt.date.today()
cl_price = pd.DataFrame()#empty dataframe to be filled with data prices
ohlcv_data = {} # empty dictionary
# looping over tickers and creating dataframe with close prices
for ticker in tickers:
    ohlcv_data[ticker] = yf.download(ticker,start,end,interval='60m')
    ohlcv_data[ticker].dropna(inplace=True,how="all")
 
tickers = ohlcv_data.keys() # redefine tickers variable after removing any tickers with corrupted data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [60]:
google = ohlcv_data['GOOG']
microsoft = ohlcv_data['MSFT']
intel = ohlcv_data['INTC']
amazon = ohlcv_data['AMZN']
sp500 = ohlcv_data['ES']

In [61]:
list(tickers)

['AMZN', 'MSFT', 'INTC', 'GOOG', 'ES']

1 = GOOG
2 = MSFT
3 = INTC
4 = AMZN
5 = ES

In [62]:
google['symbol'] = [list(tickers)[3] for x in range(len(google))]
google['symbol_coded'] = [1 for x in range(len(google))]
google.reset_index(inplace=True)
google['Datetime_coded'] = [int(str(x)[11:-6].replace(":", "")) for x in google['Datetime']]

In [63]:
google

,Datetime,Open,High,Low,Close,Adj Close,Volume,symbol,symbol_coded,Datetime_coded
0,2021-03-08 09:30:00-05:00,105.056496,106.440506,104.387497,106.097000,106.097000,398785,GOOG,1,93000
1,2021-03-08 10:30:00-05:00,106.205750,106.368996,104.815506,105.216003,105.216003,174626,GOOG,1,103000
2,2021-03-08 11:30:00-05:00,105.219749,105.233658,103.550003,104.132492,104.132492,179555,GOOG,1,113000
3,2021-03-08 12:30:00-05:00,104.051003,104.355247,103.400002,103.670494,103.670494,132624,GOOG,1,123000
4,2021-03-08 13:30:00-05:00,103.639000,103.639000,102.599998,102.878006,102.878006,141584,GOOG,1,133000
...,...,...,...,...,...,...,...,...,...,...
3363,2023-02-02 11:30:00-05:00,108.089996,108.260002,107.080002,107.120003,107.120003,4027011,GOOG,1,113000
3364,2023-02-02 12:30:00-05:00,107.129997,107.750000,106.930000,107.749397,107.749397,3550012,GOOG,1,123000
3365,2023-02-02 13:30:00-05:00,107.750000,108.660004,107.566498,107.739998,107.739998,4582997,GOOG,1,133000
3366,2023-02-02 14:30:00-05:00,107.730003,108.389999,106.809998,108.269997,108.269997,4680846,GOOG,1,143000


In [86]:
X = google[['Open', 'High', 'Low', 'Volume', 'Datetime_coded']]
y = google['Close']


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

rf = RandomForestRegressor(
    random_state=42, 
    n_estimators = 10000,
     criterion = "squared_error", max_depth: Any | None = None, min_samples_split: int = 2, min_samples_leaf: int = 1, min_weight_fraction_leaf: float = 0, max_features: float = 1, max_leaf_nodes: Any | None = None, min_impurity_decrease: float = 0, bootstrap: bool = True, oob_score: bool = False, n_jobs: Any | None = None, random_state: Any | None = None, verbose: int = 0, warm_start: bool = False, ccp_alpha: float = 0, max_samples: Any
)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

MSE = mean_squared_error(y_test, y_pred)
MAE = mean_absolute_error(y_test, y_pred)
print(MSE) # 0.14233870635516643
print(MAE) # 0.2659159655228926

# hyper-parameters: 
# -----------------
#   random_state=42, 
#   n_estimators=10000

rf2 = RandomForestRegressor()


0.14233870635516643
0.2659159655228926


n_estimators=10000, 
    criterion='squared_error', 
    max_depth=100, 
    min_samples_split=2, 
    min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, 
    max_features='log2', 
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0, 
    bootstrap=True, 
    oob_score=False, 
    n_jobs=None, 
    verbose=0, 
    warm_start=False

In [65]:
stocks_data_combined = pd.concat([google, microsoft, intel, amazon, sp500])

In [66]:
stocks_data_combined

,Datetime,Open,High,Low,Close,Adj Close,Volume,symbol,symbol_coded,Datetime_coded
0,2021-03-08 09:30:00-05:00,105.056496,106.440506,104.387497,106.097000,106.097000,398785,GOOG,1.0,93000.0
1,2021-03-08 10:30:00-05:00,106.205750,106.368996,104.815506,105.216003,105.216003,174626,GOOG,1.0,103000.0
2,2021-03-08 11:30:00-05:00,105.219749,105.233658,103.550003,104.132492,104.132492,179555,GOOG,1.0,113000.0
3,2021-03-08 12:30:00-05:00,104.051003,104.355247,103.400002,103.670494,103.670494,132624,GOOG,1.0,123000.0
4,2021-03-08 13:30:00-05:00,103.639000,103.639000,102.599998,102.878006,102.878006,141584,GOOG,1.0,133000.0
...,...,...,...,...,...,...,...,...,...,...
2023-02-02 11:30:00-05:00,NaT,83.584999,83.910004,83.540001,83.769997,83.769997,137141,NaN,NaN,NaN
2023-02-02 12:30:00-05:00,NaT,83.769997,83.870003,83.339996,83.339996,83.339996,124530,NaN,NaN,NaN
2023-02-02 13:30:00-05:00,NaT,83.360001,83.589996,83.040001,83.154999,83.154999,155663,NaN,NaN,NaN
2023-02-02 14:30:00-05:00,NaT,83.160004,83.305000,82.839996,83.305000,83.305000,180852,NaN,NaN,NaN
